# Unit tests for hapCON
WIth Paths for UoC Server

In [1]:
### Some Code to set right paths (checks whether on Chicago cluster)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os as os
import sys as sys
import multiprocessing as mp
import sys
import socket as socket

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
     path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
     print("Midway jnovmbre partition detected.")
     path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
     raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

### If wanting to use local version and not  pip installed version
### Comment out to test installed version. Comment in to test development version:
### 


# Comment in if testing Developer Package
#sys.path.insert(0,"./package/")  # hack to get local UoC Development package first in path
#sys.path.insert(0, "/mnt/archgen/users/yilei/tools/hapROH/package")  # Yilei's Development Version at MPI EVA

from hapsburg.PackagesSupport.hapsburg_run import hapsb_ind  # To test import

midway2-0401.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


# Test for 1240k panel on a Sardinian

In [2]:
# data preparation.
path2bam="./Notebooks/Vignettes/Data/SUA001.bam"
path2bed1240k="./Notebooks/Vignettes/Data/chrX_1240k.bed"
outpath="./Notebooks/Vignettes/Data/SUA001.mpileup"

!module load samtools; samtools index $path2bam
!module load samtools; samtools mpileup --positions $path2bed1240k -r X -q 30 -Q 30 -o $outpath $path2bam

[mpileup] 1 samples in 1 input files


In [3]:
from hapsburg.PackagesSupport.hapsburg_run import hapCon_chrom_BFGS

path2ref1240k="./Notebooks/Vignettes/Data/chrX_1240k.hdf5"
path2meta="./Notebooks/Vignettes/Data/meta_df_all.csv"

In [4]:
%%time 
conmle, _, _, _ = hapCon_chrom_BFGS(iid="SUA001", mpileup=outpath,
    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.1 and conmle < 0.105)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
number of major reads at flanking sites: 10636
number of minor reads at flanking sites: 16
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001502
err rate at focal sites: 0.020486
saving sample as SUA001 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001502
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading mpileup file, takes 2.024.
number of sites covered by at least one read: 3999
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102113(0.076802 - 0.127424)
CPU times: user 10.2 s, sys: 6.33 s, total: 16.6 s
Wall time: 19.4 s


### Test Running directly from bam file

In [5]:
%%time
# test running directly from bam file
conmle, _, _, _ = hapCon_chrom_BFGS(iid="SUA001", bam=path2bam,
                                    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.1 and conmle < 0.105)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 14755
number of major reads at flanking sites: 10521
number of minor reads at flanking sites: 15
number of major reads at focal sites: 1291
number of minor reads at focal sites: 27
err rate at flanking sites: 0.001424
err rate at focal sites: 0.020486
saving sample as SUA001 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated genotyping error by flanking sites: 0.001424
number of sites covered by at least one read: 3999, fraction covered: 0.085
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
finished reading bam file, takes 63.976.
number of sites covered by at least one read: 3999
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/SUA001.hdf5
estimated contamination rate: 0.102190(0.076878 - 0.127501)
CPU times: user 1min 6s, sys: 9.58 s, total: 1min 16s
Wall time: 1m

# 2) Test 1000G panel with XiongNu DA43 sample

In [6]:
%%time
path2bam="./Notebooks/Vignettes/Data/DA43.bam"
path2bed1kg="./Notebooks/Vignettes/Data/chrX_1000G.bed"
!module load samtools; samtools index $path2bam
!module load samtools; samtools mpileup --positions $path2bed1kg -r X -q 30 -Q 30 -o ./Data/DA43.mpileup $path2bam

[mpileup] 1 samples in 1 input files
CPU times: user 31 ms, sys: 32.1 ms, total: 63.1 ms
Wall time: 1.89 s


In [21]:
%%time

conmle, _, _, _ = hapCon_chrom_BFGS(iid="DA43", bam=path2bam,
                                    h5_path1000g = path2ref1240k, meta_path_ref = path2meta)
assert(conmle > 0.02 and conmle < 0.025)

exclude 1033 sites outside the specified region
exclude 0 non-SNP sites
total number of mapped reads: 281907
number of major reads at flanking sites: 4776
number of minor reads at flanking sites: 41
number of major reads at focal sites: 569
number of minor reads at focal sites: 16
err rate at flanking sites: 0.008512
err rate at focal sites: 0.027350
saving sample as DA43 in /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
estimated genotyping error by flanking sites: 0.008512
number of sites covered by at least one read: 5074, fraction covered: 0.108
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
finished reading bam file, takes 51.532.
number of sites covered by at least one read: 5074
hdf5 file saved to /project2/jnovembre/hringbauer/HAPSBURG/Notebooks/Vignettes/Data/DA43.hdf5
estimated contamination rate: 0.021650(0.008590 - 0.034710)
CPU times: user 53.7 s, sys: 7.3 s, total: 1min
Wall time: 57.7 s
